In [ ]:
pip install glob

## 將個人班表根據順序統整成總班表

In [ ]:
import openpyxl
from openpyxl import load_workbook
import glob  # 导入glob库
# 建立新的工作簿
workbook = openpyxl.Workbook()

# 選取預設的工作表
sheet = workbook.active

# 設定第一行的內容
header_row = [
    2023, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
    11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
    21, 22, 23, 24, 25, 26, 27, 28, 29, 30
]
# 將第一行內容寫入工作表
sheet.append(header_row)

# 設定第二行的內容，星期幾的文本內容
weekdays_row = [
    11, "三", "四", "五", "六",
    "日", "一", "二", "三", "四", "五", "六", "日", "一",
    "二", "三", "四", "五", "六", "日", "一", "二", "三",
    "四", "五", "六", "日", "一"
]
sheet.append(weekdays_row)

#---------建立名字list的順序--------------
name_order =  []


total_class_schedule = []

for name in name_order:
    # 使用glob库来查找包含指定名字部分的文件
    matching_files = glob.glob(f"*{name}*.xlsx")

    if matching_files:
        # 如果找到匹配的文件，选择第一个匹配的文件
        file_name = matching_files[0]
        try:
            wb = load_workbook(file_name)  # 加载 Excel 文件
            sheet0 = wb.active
            third_row = [cell.value for cell in sheet0[3]]  # 获取第三行数据并转换为列表
            sheet.append(third_row)  # 匯入 + append 個人班表到總班表中
        except FileNotFoundError:
            print(f"文件 {file_name} 未找到.")
    else:
        print(f"未找到包含'{name}'的文件.")


# 儲存工作簿為指定的檔案名稱
workbook.save("11月總班表(無下拉式選單).xlsx")

print("已成功建立並儲存測試版總班表(無下拉式選單).xlsx")


## 添加下拉式選單

In [6]:
import openpyxl
from openpyxl.worksheet.datavalidation import DataValidation

#-----------匯入檔案-------------

wb = openpyxl.load_workbook('11月總班表(無下拉式選單).xlsx')
sheet = wb.active
my_list = []
none_list = []

#----------儲存空格位置----------

for i in range(1, sheet.max_row+1):
    for j in range(1, sheet.max_column+1):
        # print(sheet.cell(row = i, column = j).value)
        if sheet.cell(row = i, column = j).value is None or sheet.cell(row = i, column = j).value == " ":
            my_list.append([i, j])

#----------將位置資訊轉成xlsx可以用的英文位置資訊----------

def number_to_excel_column(num):
    result = ""
    while num > 0:
        num -= 1
        result = chr(num % 26 + 65) + result
        num //= 26
    return result


for sublist in my_list:
    i, j = sublist
    letter = number_to_excel_column(j)
    none_list.append(f"{letter}{i}")

#----------利用上面儲存好了的位置資訊添加下拉式選單-------------

name_options = '"國內旅遊, 國外旅遊, 員工旅遊, 元旦, 228, 國慶, 清明, 端午, 中秋, 過年, 兒童, 勞動,上課, 婚假, 喪假, 病假, 公假, 產假, 事假, 育嬰假, 家庭照顧假,OFF, 102, 103, 105,AM, 207, 208, 215, PM, 公A, 315, 公P,73, 73\', 95, 85, 128, 128訪視, 210, E, E小夜當家, N, D, D°, PM°, OFF°, 271, 護, OFF+, 體檢支援,210+, N+, E+, 128+,210+, N+, E+, 128+,OFF+, 275, 262, D\', 8054,D°, PM°, OFF°"'
name = DataValidation(type='list', formula1=name_options, allow_blank=True)
sheet.add_data_validation(name)

for i in none_list:
    name.add(f'{i}')

#---------匯出檔案----------
wb.save('11月總班表(有下拉選單).xlsx')


In [7]:
df = pd.read_excel('測試版總班表(有下拉選單).xlsx')
df

NameError: name 'pd' is not defined

# 總班表隨機生成

In [ ]:
import pandas as pd
import numpy as np
import os

# 輸出存放位址
output_directory = 'output_data'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
# 讀file
excel_file_path = '112年10月總班表(含下拉選單).xlsx'
df = pd.read_excel(excel_file_path)

# 第一列和其他列
col_to_lock = df.columns[0]
cols_to_randomize = df.columns[1:]

# 隨機數據
for i in range(200):
    # 複製隨機數據
    df_cols_copy = df[cols_to_randomize].copy()

    df_cols_copy = df_cols_copy.apply(np.random.permutation, axis=0)
   
    df[cols_to_randomize] = df_cols_copy

    output_path = f'output_data/output_{i + 1}.csv'
    df.to_csv(output_path, index=False)
df[col_to_lock] = df[col_to_lock]

df.to_csv('output_data/original_data.csv', index=False)